In [1]:
#import the standard libraries
import pandas as pd, numpy as np

#import the model learning libraries
from sklearn import feature_selection
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics

#### Описательный Анализ Данных

In [2]:
#import the data set
data = pd.read_excel('data/data_ford_price.xlsx')

#read the data set
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [3]:
#the total number of rows and columns
data.shape

(7017, 12)

In [4]:
#basic information on the data set
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


#### Преобразование Данных

In [5]:
#check for null values
data.isnull().sum()[data.isnull().sum() > 0]

drive       391
size       1564
weather     180
dtype: int64

In [6]:
#delete the null values
data = data.dropna()

#### Обучение Модели

In [7]:
#feature selection
numerical_features = ['year', 'cylinders', 'odometer', 'lat', 'long', 'weather']

X = data[numerical_features]
y = data['price']

In [8]:
#divide the data into train and test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

In [9]:
#create an object class
lr_model = linear_model.LinearRegression()

#model learning
lr_model.fit(X_train, y_train)

#make a prediction
y_test_pred = lr_model.predict(X_test)

#display the metrics
print(f'MAE: {np.round(metrics.mean_absolute_error(y_test, y_test_pred), 3)}')
print(f'R2: {np.round(metrics.r2_score(y_test, y_test_pred), 3)}')

MAE: 4620.233
R2: 0.634


#### Recursive Feature Elimination (RFE) for Feature Selection

In [10]:
#create the object class
lr_model_rfe = linear_model.LinearRegression()

#find the best features using RFE
selector_rfe = feature_selection.RFE(lr_model_rfe,
                                     n_features_to_select=3,
                                     step=1)

#model learning
selector_rfe.fit(X_train, y_train)

#display the selected features
rfe_columns = list(selector_rfe.get_feature_names_out())
rfe_columns

['year', 'cylinders', 'lat']

In [11]:
#regression learning on RFE
X_train_rfe = X_train[rfe_columns]
X_test_rfe = X_test[rfe_columns]

#create an object class
lr_model = linear_model.LinearRegression()

#model learning
lr_model.fit(X_train_rfe, y_train)

#make a prediction
y_test_pred = lr_model.predict(X_test_rfe)

#display the metrics
print(f'MAE: {np.round(metrics.mean_absolute_error(y_test, y_test_pred), 3)}')
print(f'R2: {np.round(metrics.r2_score(y_test, y_test_pred), 3)}')

MAE: 5107.525
R2: 0.573


#### SelectKBest for Feature Selection

In [12]:
#find the best features using KBest
selector_kbest = feature_selection.SelectKBest(feature_selection.f_regression,
                                               k=3)

#model learning
selector_kbest.fit(X_train, y_train)

#display the selected features
kbest_columns = list(selector_kbest.get_feature_names_out())
kbest_columns

['year', 'cylinders', 'odometer']

In [13]:
#regression learning on KBest
X_train_kbest = X_train[kbest_columns]
X_test_kbest = X_test[kbest_columns]

#create an object class
lr_model = linear_model.LinearRegression()

#model learning
lr_model.fit(X_train_kbest, y_train)

#make a prediction
y_test_pred = lr_model.predict(X_test_kbest)

#display the metrics
print(f'MAE: {np.round(metrics.mean_absolute_error(y_test, y_test_pred), 3)}')
print(f'R2: {np.round(metrics.r2_score(y_test, y_test_pred), 3)}')

MAE: 4627.369
R2: 0.631


#### Какой метод отбора признаков показал наилучший результат на тестовой выборке?

**Вывод:**

Было построено три модели; с помощью методов RFE и KBest были выбраны лучшие признаки для линейной регрессии:
- базовая модель линейной регрессии на числовых признаках;
- модель линейной регрессии для признаков - year, cylinders, lat - выбранные с помощью метода RFE;
- модель линейной регрессии для признаков - year, cylinders, odometer - выбранные с помощью метода KBest

Качество моделей сравнивались по значениям Mean Absolute Error (MAE) и Оценки R2.

|   | Linear Regression | Linear Regression with RFE | Linear Regression with KBest |
|:-:| :---------------: | :------------------------: | :--------------------------: |
MAE | 4620.2            | 5107.5                     | 4627.4                       |
R2  | 0.634             | 0.573                      | 0.631                        |

Средняя абсолютная ошибка измеряется как средняя абсолютная разница между прогнозируемыми значениями и фактическими значениями; чем ближе к 0, тем лучше. \
R² или Коэффициент Детерминации определяет долю дисперсии зависимой переменной, которую можно объяснить независимой переменной, то есть показывает, насколько хорошо данные соответствуют степень соответствия, чем ближе к 1, тем лучше.

Использование линейной регрессии для всех числовых признаков показывает наилучшие результаты в отношении MAE и R2, однако использование большего количества признаков в машинном обучении не всегда является лучшим подходом и может привести к переобучению, и модель может работать хуже с другим набором данных.

Если небольшой размер признаков обеспечивает такую ​​же точность, как и модель с большим количеством признаков - это всегда полезнее, поскольку производительность увеличивается с точки зрения более быстрого получения результатов. \
Таким образом, если результаты третьей модели, в которой используется лучший выбор признаков метода KBest, сравнить с базовой моделью линейной регрессии, различия между MAE и R2 довольно малы, и в работе с другим набором данных лучше использовать третью модель.

Если результаты третьей модели сравнить со второй моделью, в которой используется лучший выбор признаков метода RFE, то результаты второй модели значительно хуже по показателям MAE и R2: разница между прогнозируемыми значениями и фактическими значениями выше, а доля дисперсии ниже. \
Скорее всего это происходит потому, что RFE итеративно исключает по одному слабейшему признаку за каждый цикл, а KBest считывает результаты по всем признакам за раз и выбирает сильнейшие.